In [ ]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
openai_client = OpenAI(api_key=os.environ['API_KEY'])

# Tavily APIクライアントを生成
tavily_client = TavilyClient(api_key=os.environ['TAVILY_API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [ ]:
def define_tools():
    """
    OpenAIのAPIに渡すツール定義を作成する。

    Returns:
        ツール定義のリスト
    """
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]


def get_search_result(question):
    """
    TavilyでWeb検索を実行し、検索結果をJSON文字列として返す関数。

    Args:
        検索に使う質問文。

    Returns:
        整形したJSON文字列。
    """
    response = tavily_client.search(question)
    return json.dumps({"result": response["results"]})

In [ ]:
def ask_question(messages, tools=None, tool_choice=None):
    """
    OpenAIのAPIを呼び出して、言語モデルに質問を行う関数。

    Args:
        messages (list[dict]):
            OpenAIのmessages形式の配列。
        tools:
            OpenAI APIのtoolsパラメータ。
        tool_choice:
            ツールをどう使うかの指定。

    Returns:
        APIのレスポンスオブジェクト。
    """
    kwargs = {
        "model": MODEL_NAME,
        "messages": messages,
    }
    if tools is not None:
        kwargs["tools"] = tools
    if tool_choice is not None:
        kwargs["tool_choice"] = tool_choice

    response = openai_client.chat.completions.create(**kwargs)
    return response


def execute_tool_call(tool):
    """
    tool_callsで指定された関数を実行する関数。

    Args:
        tool: モデルが要求したツール呼び出し情報

    Returns:
        呼び出した関数の戻り値（文字列）
    """
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)
    return function_response


def handle_tool_call(response, messages):
    """
    ツール呼び出しが必要な場合の処理を行う関数。

    Args:
        response:
            ask_question() の戻り値。
        messages (list[dict]):
            これまでの会話履歴。

    Returns:
        list[dict]:
            tool実行結果を追加したmessages。
    """
    work_messages = list(messages)
    work_messages.append(response.choices[0].message)

    for tool in response.choices[0].message.tool_calls:
        function_response = execute_tool_call(tool)

        work_messages.append(
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            }
        )

    return work_messages


def trim_messages(messages, max_len):
    """
    履歴メッセージを制限する関数。
    会話履歴を最大 max_len 件に制限する（systemは先頭に保持する）。

    Args:
        messages (list[dict]):
            会話履歴。
        max_len (int):
            systemを除いて残す最大件数。

    Returns:
        list[dict]:
            制限後の履歴メッセージ。
    """
    has_system = bool(messages) and messages[0].get("role") == "system"
    start_index = 1 if has_system else 0

    system_messages = messages[:start_index]
    other_messages = messages[start_index:]

    messages[:] = system_messages + other_messages[-max_len:]
    return messages


def process_response(messages, tools):
    """
    ユーザーからの質問を処理する関数。

    Args:
        messages (list[dict]):
            会話履歴。
        tools:
            Function calling用のツール定義。

    Returns:
        str:
            ユーザーからの質問の回答テキスト。
    """
    response = ask_question(messages, tools=tools, tool_choice="auto")

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        work_messages = handle_tool_call(response, messages)
        final_response = ask_question(work_messages)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()


# チャットボットへの組み込み
tools = define_tools()

messages=[]

# キャラクター設定
SYSTEM_PROMPT = "あなたは猫のキャラクターとして振る舞うチャットボットです。語尾に「にゃ」を付け、明るく親しみやすく返答してください。"

# システムプロンプトを先頭に追加
messages.append({"role": "system", "content": SYSTEM_PROMPT})

try:
    while(True):
        # ユーザーからの質問を受付
        question = input("メッセージを入力:")
        # 質問が入力されなければ終了
        if question.strip()=="":
            break
        display(f"質問:{question}")

        # メッセージにユーザーからの質問を追加
        messages.append({"role": "user", "content": question.strip()})
        # やりとりが8を超えたら古いメッセージから削除
        messages = trim_messages(messages, 8)

        # 言語モデルに質問
        response_message = process_response(messages, tools)

        # メッセージに言語モデルからの回答を追加
        print(response_message, flush=True)
        messages.append({"role": "assistant", "content": response_message})

except KeyboardInterrupt:
    print("処理を中断しました。")

print("---ご利用ありがとうございました！---")